<a href="https://colab.research.google.com/github/anthonyluosen/NLP_transformer/blob/main/task7_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import random
import numpy as  np

seed  = 11
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [4]:
pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 30.7 MB/s 
     |████████████████████████████████| 895 kB 34.5 MB/s 
     |████████████████████████████████| 636 kB 37.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [8]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [9]:
from torchtext.legacy import data
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [10]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(seed))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 30.6MB/s]


In [11]:
train_data, valid_data = train_data.split(random_state = random.seed(seed))

In [12]:
print(vars(train_data.examples[6]))


{'text': [1045, 4033, 1005, 1056, 4191, 2023, 2524, 2012, 1037, 3185, 1999, 1037, 2146, 2051, 1012, 1045, 2288, 2000, 2175, 2000, 2019, 5083, 11326, 1010, 1998, 2001, 16082, 2138, 1045, 2018, 2042, 5996, 2000, 2156, 2009, 1012, 1045, 2018, 4000, 1999, 2026, 2159, 2013, 7239, 2802, 1037, 2843, 1997, 1996, 3185, 1012, 1996, 4378, 2035, 4207, 2026, 7239, 1010, 1998, 2001, 27104, 1998, 13175, 2802, 2087, 1997, 1996, 3185, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 13970, 12269, 2000, 3889, 12385, 5349, 1006, 2040, 1045, 2018, 2525, 2042, 1037, 5470, 1997, 1007, 1012, 2002, 16481, 1999, 2023, 3185, 2010, 14388, 5848, 2005, 4038, 1012, 2002, 2038, 1037, 2806, 2008, 1045, 4033, 1005, 1056, 2464, 2077, 1012, 1998, 6615, 10326, 2121, 2003, 6581, 2004, 2467, 1012, 4067, 2643, 2045, 2347, 1005, 1056, 1037, 12081, 2013, 2097, 10768, 14069, 1006, 2293, 2032, 1010, 2021, 2387, 2032, 2205, 2172, 2023, 2621, 1007, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2045, 2020, 3033, 1997,

In [13]:
print(tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text']))


['i', 'haven', "'", 't', 'laughed', 'this', 'hard', 'at', 'a', 'movie', 'in', 'a', 'long', 'time', '.', 'i', 'got', 'to', 'go', 'to', 'an', 'advance', 'screening', ',', 'and', 'was', 'thrilled', 'because', 'i', 'had', 'been', 'dying', 'to', 'see', 'it', '.', 'i', 'had', 'tears', 'in', 'my', 'eyes', 'from', 'laughter', 'throughout', 'a', 'lot', 'of', 'the', 'movie', '.', 'the', 'audience', 'all', 'shared', 'my', 'laughter', ',', 'and', 'was', 'clapping', 'and', 'yelling', 'throughout', 'most', 'of', 'the', 'movie', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'ku', '##dos', 'to', 'steve', 'carr', '##ell', '(', 'who', 'i', 'had', 'already', 'been', 'a', 'fan', 'of', ')', '.', 'he', 'proves', 'in', 'this', 'movie', 'his', 'tremendous', 'talent', 'for', 'comedy', '.', 'he', 'has', 'a', 'style', 'that', 'i', 'haven', "'", 't', 'seen', 'before', '.', 'and', 'catherine', 'keen', '##er', 'is', 'excellent', 'as', 'always', '.', 'thank', 'god', 'there', 'wasn', "'", 't', 'a', 'cameo', 'from', 

In [14]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'pos': 0, 'neg': 1})


In [15]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [16]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
import torch.nn as nn
class BertRNNSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, bidirectional, hidden_layers, dropout):
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.gru = nn.GRU(embedding_dim,
                         hidden_dim,
                         num_layers = hidden_layers , 
                         bidirectional = bidirectional , 
                         batch_first = True , 
                         dropout = 0 if hidden_layers<2  else dropout)
        self.fc  = nn.Linear(hidden_dim*2 if bidirectional else hidden_dim ,output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self , x):
        # text = [batch size , sent len]
        with torch.no_grad():
            embedded = self.bert(x)[0]
        # [batch size , sent len , embedding dim]
        _ , hidden = self.gru(embedded)  # output hidden 
        #[n_layers*n directional  ,batch size , hidden dim]
        if self.gru.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:] , hidden[-1,:,:]) ,dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #  [batch size , hidden dim]
        out  = self.fc(hidden)
        # [ batch size , output dim]
        return out

In [18]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BertRNNSentiment(bert, HIDDEN_DIM, OUTPUT_DIM, BIDIRECTIONAL, N_LAYERS, DROPOUT)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [20]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [22]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()


In [23]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

gru.weight_ih_l0
gru.weight_hh_l0
gru.bias_ih_l0
gru.bias_hh_l0
gru.weight_ih_l0_reverse
gru.weight_hh_l0_reverse
gru.bias_ih_l0_reverse
gru.bias_hh_l0_reverse
gru.weight_ih_l1
gru.weight_hh_l1
gru.bias_ih_l1
gru.bias_hh_l1
gru.weight_ih_l1_reverse
gru.weight_hh_l1_reverse
gru.bias_ih_l1_reverse
gru.bias_hh_l1_reverse
fc.weight
fc.bias


In [34]:
model = model.to(device)
criterion = criterion.to(device)
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [25]:
device

device(type='cuda')

In [35]:
# def train(model , train_iterator , optimizer , criterion):
#     epoch_loss = 0
#     epoch_acc = 0
#     model.train()
#     for batch in train_iterator:
#         optimizer.zero_grad()
#         predictions = model(batch.text).squeeze(1)
#         loss = criterion(predictions , batch.label)
#         acc = binary_accuracy(predictions , batch.label)
#         loss.backward()
#         optimizer.step()
#         epoch_loss+=loss.item()
#         epoch_acc +=acc.item()
#         return epoch_loss/len(train_iterator) , epoch_acc/len(train_iterator)
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
# def evaluate(model,valid_iterator , criterion):
#     epoch_loss = 0
#     epoch_acc = 0
#     model.eval()
#     with torch.no_grad():
#         for batch in valid_iterator:
#             predictions = model(batch.text).squeeze(1)

#             loss = criterion(predictions , batch.label)
#             acc = binary_accuracy(predictions , batch.label)
#             epoch_loss+=loss.item()
#             epoch_acc+=acc.item()
#     return epoch_loss/len(valid_iterator) , epoch_acc/len(valid_iterator)
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 17m 47s
	Train Loss: 0.553 | Train Acc: 69.96%
	 Val. Loss: 0.423 |  Val. Acc: 82.81%
Epoch: 02 | Epoch Time: 17m 47s
	Train Loss: 0.304 | Train Acc: 87.16%
	 Val. Loss: 0.264 |  Val. Acc: 89.36%
Epoch: 03 | Epoch Time: 17m 47s
	Train Loss: 0.249 | Train Acc: 89.92%
	 Val. Loss: 0.282 |  Val. Acc: 89.16%
Epoch: 04 | Epoch Time: 17m 47s
	Train Loss: 0.217 | Train Acc: 91.59%
	 Val. Loss: 0.243 |  Val. Acc: 90.66%
Epoch: 05 | Epoch Time: 17m 47s
	Train Loss: 0.186 | Train Acc: 92.71%
	 Val. Loss: 0.253 |  Val. Acc: 91.05%


# 新段落